In [ ]:
# import necessary packages
import numpy as np
import matplotlib.pyplot as plt

### Sklearn:

In [ ]:
# fetching LogisticRegression and dataset rom sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler

digits = load_digits()
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=0)
logReg = LogisticRegression()

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)  

logReg.fit(x_train,y_train)

In [ ]:
predictions = logReg.predict(x_test)
score = logReg.score(x_test, y_test)
print(score)

### Own code:

In [ ]:
from logisticReg import SoftmaxReg 
from gdOptimizers import GDOptimizers

In [ ]:
digits = load_digits()
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=0)

image, label = digits.data[0], digits.target[0]
image = image.reshape((8,8))
plt.imshow(image, cmap='gray')
print('Label:', label)

In [ ]:
model = SoftmaxReg(64,10,0)
sgd = GDOptimizers(learning_rate = 0.01, epochs = 1000, batch_size = 10, optimizer = "SGD")
print(x_train.shape)
model.fit(x_train, y_train, sgd)

In [ ]:
weights = np.random.randn(64, 10)
bias = np.zeros([10,1]) + 0.01
theta = np.array([weights, bias], dtype=object)